In [0]:
%sql
USE CATALOG pricing_analytics;

INSERT INTO silver.daily_pricing_silver
SELECT 
    to_date(DATE_OF_PRICING, 'MM/dd/yyyy') AS date_of_pricing,
    try_cast(ROW_ID AS BIGINT) AS row_id,
    NULLIF(TRIM(STATE_NAME), '') AS STATE_NAME,
    NULLIF(TRIM(MARKET_NAME), '') AS MARKET_NAME,
    NULLIF(TRIM(PRODUCTGROUP_NAME), '') AS PRODUCTGROUP_NAME,
    NULLIF(TRIM(PRODUCT_NAME), '') AS PRODUCT_NAME,
    NULLIF(TRIM(VARIETY), '') AS VARIETY,
    NULLIF(TRIM(ORIGIN), '') AS ORIGIN,
    try_cast(ARRIVAL_IN_TONNES AS DECIMAL(18, 2)) AS arrival_in_tonnes,
    try_cast(MINIMUM_PRICE AS DECIMAL(36, 2)) AS minimum_price,
    try_cast(MAXIMUM_PRICE AS DECIMAL(36, 2)) AS maximum_price,
    try_cast(MODAL_PRICE AS DECIMAL(36, 2)) AS modal_price,
    source_file_load_date, -- date when files are loaded to bronze layer
    CURRENT_TIMESTAMP() AS inserted_at, -- date when files are inserted to silver layer
    CURRENT_TIMESTAMP() AS updated_at -- date when files are updated to silver layer
FROM pricing_analytics.bronze.daily_pricing
WHERE source_file_load_date > (
    SELECT NVL(MAX(PROCESSED_TABLE_DATETIME), '1900-01-01') 
    FROM pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS
    WHERE process_name = 'daily_pricing_silver' 
      AND process_status = 'Completed'
)
AND try_cast(ROW_ID AS BIGINT) IS NOT NULL
AND try_cast(ARRIVAL_IN_TONNES AS DECIMAL(18, 2)) IS NOT NULL
AND try_cast(MINIMUM_PRICE AS DECIMAL(36, 2)) IS NOT NULL
AND try_cast(MAXIMUM_PRICE AS DECIMAL(36, 2)) IS NOT NULL
AND try_cast(MODAL_PRICE AS DECIMAL(36, 2)) IS NOT NULL

In [0]:
# %sql
# MERGE INTO silver.daily_pricing_silver AS tgt
# USING (
#   SELECT 
#     to_date(DATE_OF_PRICING, 'dd/MM/yyyy') AS date_of_pricing,
#     CAST(ROW_ID AS BIGINT) AS row_id,
#     STATE_NAME,
#     MARKET_NAME,
#     PRODUCTGROUP_NAME,
#     PRODUCT_NAME,
#     VARIETY,
#     ORIGIN,
#     CAST(ARRIVAL_IN_TONNES AS DECIMAL(18, 2)) AS arrival_in_tonnes,
#     CAST(MINIMUM_PRICE AS DECIMAL(36, 2)) AS minimum_price,
#     CAST(MAXIMUM_PRICE AS DECIMAL(36, 2)) AS maximum_price,
#     CAST(MODAL_PRICE AS DECIMAL(36, 2)) AS modal_price,
#     source_file_load_date,
#     current_timestamp() AS _now -- dùng lại cho cả insert và update
#   FROM pricing_analytics.bronze.daily_pricing
#   WHERE source_file_load_date > (
#     SELECT NVL(MAX(PROCESSED_TABLE_DATETIME), '1900-01-01')
#     FROM pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS
#     WHERE process_name = 'daily_pricing_silver'
#       AND process_status = 'Completed'
#   )
# ) AS src
# ON tgt.row_id = src.row_id
# WHEN MATCHED THEN UPDATE SET
#   tgt.date_of_pricing = src.date_of_pricing,
#   tgt.state_name = src.state_name,
#   tgt.market_name = src.market_name,
#   tgt.productgroup_name = src.productgroup_name,
#   tgt.product_name = src.product_name,
#   tgt.variety = src.variety,
#   tgt.origin = src.origin,
#   tgt.arrival_in_tonnes = src.arrival_in_tonnes,
#   tgt.minimum_price = src.minimum_price,
#   tgt.maximum_price = src.maximum_price,
#   tgt.modal_price = src.modal_price,
#   tgt.source_file_load_date = src.source_file_load_date,
#   tgt.updated_at = src._now

# WHEN NOT MATCHED THEN INSERT (
#   date_of_pricing,
#   row_id,
#   state_name,
#   market_name,
#   productgroup_name,
#   product_name,
#   variety,
#   origin,
#   arrival_in_tonnes,
#   minimum_price,
#   maximum_price,
#   modal_price,
#   source_file_load_date,
#   inserted_at,
#   updated_at
# )
# VALUES (
#   src.date_of_pricing,
#   src.row_id,
#   src.state_name,
#   src.market_name,
#   src.productgroup_name,
#   src.product_name,
#   src.variety,
#   src.origin,
#   src.arrival_in_tonnes,
#   src.minimum_price,
#   src.maximum_price,
#   src.modal_price,
#   src.source_file_load_date,
#   src._now,
#   src._now
# );


In [0]:
%sql
INSERT INTO pricing_analytics.processrunlogs.deltalakehouse_process_runs(`PROCESS_NAME`, `PROCESSED_TABLE_DATETIME`, `PROCESS_STATUS`)
SELECT 'daily_pricing_silver', max(source_file_load_date), 'Completed' 
FROM pricing_analytics.silver.daily_pricing_silver